In [2]:
import os
from shutil import copyfile
from tqdm import tqdm
import csv
from datetime import datetime
import random 

In [6]:
def flare_or_not(peak_flux):
    if peak_flux >= 1e-5:
        return True
    else:
        return False

In [4]:
def read_csv(csv_path):
    meta_data = []
    first_time = True
    #path + 'meta_data.csv'

    with open(csv_path) as csvfile:
        readCSV = csv.reader(csvfile, delimiter=',')
        for row in readCSV:
            if first_time:
                first_time = False
                continue
            meta_data.append([row[0], datetime.strptime(str(row[1])[:-3], '%Y-%m-%d %H:%M:%S.%f'), datetime.strptime(str(row[2])[:-3], '%Y-%m-%d %H:%M:%S.%f'), float(row[3]), flare_or_not(float(row[3]))])
    return meta_data

In [7]:
def get_file_name_list(csv_path):
    meta_data = read_csv(csv_path)
    file_name_list = []
    for entry in meta_data:
        file_name_list.append([str(entry[1].year) + '-' + str('{:02d}'.format(entry[1].month)) + '-' + str('{:02d}'.format(entry[1].day)) + 'T' + str('{:02d}'.format(entry[1].hour)) + str('{:02d}'.format(entry[1].minute)) + str('{:02d}'.format(entry[1].second)) + '__magnetogram.jpg', entry[4]])
        
    return file_name_list

In [9]:
paths = ['../../../solarflare-catcher/sdobenchmark_full/test/', '../../../solarflare-catcher/sdobenchmark_full/training/']

flare_counter = 0
no_flare_counter = 0

flares = []
no_flares = []

for path in paths:
    magnetogram_file_names = get_file_name_list(path + 'meta_data.csv')
    for root,d_names,f_names in tqdm(os.walk(path)):
        for file in f_names:
            if 'magnetogram' in file and [file, True] in magnetogram_file_names:
                flares.append([file, root])
            elif 'magnetogram' in file and [file, False] in magnetogram_file_names:
                no_flares.append([file, root])

balanced_data_size = min([len(flares), len(no_flares)])
            
for t in flares:
    copyfile(t[1] + '/' + t[0], './dataset/flare/' + str(flare_counter) + '.' + t[0].split('.')[-1])
    flare_counter = flare_counter + 1

random.shuffle(no_flares)

for f in no_flares:
    if no_flare_counter < balanced_data_size:
        copyfile(f[1] + '/' + f[0], './dataset/no_flare/' + str(no_flare_counter) + '.' + f[0].split('.')[-1])
        no_flare_counter = no_flare_counter + 1
    else:
        break


979it [00:00, 4944.39it/s]
9429it [00:12, 730.97it/s]
